In [1]:
import os
from urllib import request

from mmcv import Config

from utils import modify_path, modify_num_classes

## Model Name / Config name / Checkpoint URL

In [2]:
model_name = \
"detectors"

model_cfg = \
"DetectoRS_mstrain_400_1200_x101_32x4d_40e"

model_url = \
"https://www.cs.jhu.edu/~syqiao/DetectoRS/DetectoRS_X101-ed983634.pth"

## Set Config Setting Values.

In [3]:
lr_gamma = 0.5

epoch_size = 15
batch_size = 4

step_milestone = [11, 14]

multi_scale_factor = [0.5, 0.75, 1.0, 1.25, 1.5]

insta_boost_flag = True

## Load Config

In [4]:
cfg = Config.fromfile(os.path.join("./configs", model_name, f"{model_cfg}.py"))

## Modify Config

In [5]:
# modify lr_gamma * default lr.
cfg.optimizer["lr"] *= lr_gamma

# modify evaluation image scales.
if multi_scale_factor:
    test_h, test_w = cfg.data.test.pipeline[1]["img_scale"]
    cfg.data.test.pipeline[1]["img_scale"] = [(int(test_h * scale_factor), int(test_w * scale_factor)) for scale_factor in multi_scale_factor]
    cfg.data.test.pipeline[1]["flip"] = True

# add pretrained model from url.
cfg.load_from = os.path.abspath(os.path.join("pretrained", f"{model_cfg}.pth"))

# add wandb log.
cfg.log_config = dict(
    interval=50, hooks=[dict(type="TextLoggerHook"), dict(type="WandbLoggerHook", init_kwargs=dict(project="pstage-3-od", name=model_cfg))]
)

# pth model file would be saved for most recent 3 epoch.
cfg.checkpoint_config = dict(interval=1, max_keep_ckpts=3)

# validation processed for every epoch, and best bbox mAP 50 model would be saved.
cfg.evaluation = dict(interval=1, metric='bbox', save_best="bbox_mAP_50")

# modify data path, save path.
modify_path(cfg, data_path="../input/data", save_path=os.path.join("./save_dir", model_cfg))

# modify num of class.
modify_num_classes(cfg, class_num=11)

# batch size and num of worker.
cfg.data.samples_per_gpu = batch_size
cfg.data.workers_per_gpu = 4

# epoch size and step milestone.
cfg.runner["max_epochs"] = epoch_size
cfg.lr_config["step"] = step_milestone

# if checkpoint url exists, clear pretrained.
if model_url:
    cfg.model.pretrained = None

# insta boost apply
if insta_boost_flag:
    insta_boost_cfg = dict(
        type='InstaBoost',
        action_candidate=('normal', 'horizontal', 'skip'),
        action_prob=(1, 0, 0),
        scale=(0.8, 1.2),
        dx=15,
        dy=15,
        theta=(-1, 1),
        color_prob=0.5,
        hflag=False,
        aug_ratio=0.5
    )
    cfg.data.train.pipeline.insert(1, insta_boost_cfg)

## Save Config

In [6]:
os.makedirs(os.path.join("custom_configs"), exist_ok=True)
cfg.dump(os.path.join("custom_configs", f"{model_cfg}.py"))

## Pretrained Checkpoint Save.

In [7]:
if not os.path.isdir("pretrained"):
    os.makedirs("pretrained")

ckpt_name = os.path.join("pretrained", f"{model_cfg}.pth")

request.urlretrieve(model_url, ckpt_name)

('pretrained/DetectoRS_mstrain_400_1200_x101_32x4d_40e.pth',
 <http.client.HTTPMessage at 0x7f06cdacdd10>)